In [3]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [6]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

### Import and visualizing dataset

In [10]:
#import the dataset 
data_set = pd.read_csv("Iris.csv")

#Let's have a look at the dataset
data_set.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Data Visualization

### Separating Examples and Labels and string conversion to numerics

In [11]:
Species = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica':2}
data_set.Species = [Species[item] for item in data_set.Species]

X = data_set.iloc[:, 0:4] #predictors
y = data_set.iloc[:, 4]

In [12]:
from sklearn.preprocessing import normalize

X = normalize(X)
y = np.array(y)
y = y.astype(int)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.10)
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

X_train shape:  (135, 4)
Y_train shape:  (135,)
X_test shape:  (15, 4)
Y_test shape:  (15,)


### Using Dataloader to convert numpy arrays to Tensors

In [15]:
# for loading data into pytorch model
from torch.utils.data import TensorDataset, DataLoader

trainloader = DataLoader(TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train)), 
                         batch_size=135, shuffle=True)

testloader = DataLoader(TensorDataset(torch.from_numpy(X_test), torch.from_numpy(Y_test)), 
                        batch_size=135, shuffle=True)

dataloaders = {
    "train": trainloader,
    "validation": testloader
}


### This class will define our model

#### Using  init we will define number of nodes in the particular layer
#### forward() defines the functionality of each layer

In [18]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 227) ### 4 predictors
        self.fc2 = nn.Linear(227, 94) ### 227 neurons in hidden layer 1, 94 in 3
        self.fc3 = nn.Linear(94, 75) ### 75 neurons in hidden layer 3
        self.fc4 = nn.Linear(75, 3) ### 3 classes
        
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x)) #activation function for 1
        x = F.relu(self.fc2(x))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

### Model declaration, Type of loss and optimizer.

#### Adam optimizer is being used to optimize our network

In [19]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

### This block print a summary of model